

Se abren las imagenes  ya  recortadas, que solo tengan el color checker, y se tienen en formato TIF

Luego se abren estas imágenes, para que sean leídas como matrices, las componentes de estas matrices son los valores de RGB de la imagen, para entenderlo mejor se tiene que imaginar un cubo en donde la cara que se ve al frente es el R y si me paro en cada componente en esa cara me da el valor de R, ahora si voy al fondo de esta cara están los valores de G y B. Estos valores se substraen fácilmente con el comando getpixel.

Luego se recorre toda la imagen para conocer los diferentes colores, empezando por el color superior izquierdo y llegando al color inferior derecho. En cada color se para en un pixel que esta cerca del centro y este coge una cantidad de vecinos y se promedia para obtener un valor más general, luego como son varias fotos se hace un promedio total de esos valores de color que se obtiene.

Acá se manipulan dos imagenes en formato .JPG, sin embargo el formato .TIF se abre de la misma manera.



#Librerías 

In [ ]:
# -*- coding: utf-8 -*-
from matplotlib import pyplot as plt
import numpy as np
import scipy.misc as sp
from PIL import Image
#from IPython.display import Image
!pip install rawpy
import rawpy
!pip install colormath
from colormath.color_objects import sRGBColor, LabColor              #Las librerías especiales para comparar colores 
from colormath.color_conversions import convert_color                # http://hanzratech.in/2015/01/16/color-difference-between-2-colors-using-python.html
from colormath.color_diff import delta_e_cie2000

     |████████████████████████████████| 1.6MB 2.7MB/s 
  Created wheel for colormath: filename=colormath-3.0.0-cp36-none-any.whl size=39392 sha256=28b66a8f4f5e57fa73ec90843823c72f0d175a18204f6cdaff49394b7e6f3e29
  Stored in directory: /root/.cache/pip/wheels/20/49/7b/9635bccd6136cc713f623caa85ced89c0c63878f8b2a58fe99
Successfully built colormath


#Importar imagenes

In [ ]:
from google.colab import files

In [ ]:
infog = files.upload()

Saving prueba1.png to prueba1.png
Saving prueba2.png to prueba2.png


#Calibrar imágenes
Codigo que reescala las imágenes. Esto quiere decir que ya se pueden unificar las coordendas y no toca hacer eso manual. Sin embargo, las imagenes trabajadas acá, ya deben estar recortadas, de tal manera que solo quede el ColorChecker.

In [ ]:
'''
https://gist.github.com/tomvon/ae288482869b495201a0
'''

png1 = Image.open('prueba1.png')#imagen cortada
png2 = Image.open('prueba2.png')


mywidth = 4700 #tamano luego de cortar la imagen 4700 × 3332

wpercent1 = (mywidth/float(png1.size[0]))
hsize1 = int((float(png1.size[1])*float(wpercent1)))
png1 = png1.resize((mywidth,hsize1))
png1.save('prueba11r.png')#imagen calibrada


wpercent2 = (mywidth/float(png2.size[0]))
hsize2 = int((float(png2.size[1])*float(wpercent2)))
png2 = png2.resize((mywidth,hsize2))
png2.save('prueba22r.png')

#Código del análisis de las fotos

In [ ]:
png1 = Image.open('prueba11r.png')#imagenes ya cortadas y calibradas
png2 = Image.open('prueba22r.png')


unitlenght = 770 #distancia entre centros
lst1, lst2 = [], [] #definir lista donde se van a poner las tripletas de rgb de cada foto
l1, l2 = [], [] # listas para guardar los valores rgb de cada foto y luego compararlos, para obtener el promedio general

for j in range (0,4,1): #corre por cada fila (4 en total)
    for i in range (0,6,1): #corre por cada columna (6 en total) para 24 cuadros
        lenght = unitlenght*i #distancia entre centro 0 y centro cuadro i
        for a in range (430, 490, 5): #definir coordenadas columna cuadro 0
            a1 = a+lenght #correr horizontalmente del cuadro 0 -->
            for b in range(430+j*unitlenght,490+j*unitlenght,5): #definir coordenada fila cuadro 0 y correr verticalmente (abajo)
                lst1.append(png1.getpixel((a1,b))) #llenar lista con las tripletas (tuples) rgb
                lst2.append(png2.getpixel((a1,b)))
        #print(lst)
        mean1=list(np.mean(lst1, axis=0)) #promediar la lista
        mean2=list(np.mean(lst2,axis=0))
        #print('El promedio rgb041 de %d es:'%(i+j*6), mean1, a1,b) #imprime promedio y ultimas coordenadas para saber si si esta en el lugar correcto
        #print('El promedio rgb038 de %d es:'%(i+j*6), mean2, a1,b)
        l1.append(mean1)
        l2.append(mean2)
        lst1.clear() #limpiar la lista para el siguiente cuadro
        lst2.clear()

Valor teórico obtenido en wikipedia https://en.wikipedia.org/wiki/ColorChecker

In [ ]:
teorico = [[115,82,68],[194,150,130], [98,122,157],[87,108,67],[133,128,177],[103,189,170],[214,126,44],[80,91,166],[193,90,99],[94,60,108],[157,188,64],[224,163,46],[56,61,150],[70,148,73],[175,54,60],[231,199,31],[187,86,149],[8,133,161],[243,243,242],[200,200,200],[160,160,160],[122,122,121],[85,85,85],[52,52,52]]

Definir a gran lista donde se guarda el promedio de las tripletas de cada color

In [ ]:
filelist0,filelist1,filelist2, filelist3, filelist4, filelist5, filelist6, filelist7, filelist8, filelist9, filelist10, filelist11, filelist12, filelist13, filelist14, filelist15, filelist16, filelist17, filelist18, filelist19, filelist20, filelist21, filelist22, filelist23= [],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]

filelists = [filelist0,filelist1, filelist2, filelist3, filelist4, filelist5, filelist6, filelist7, filelist8, filelist9, filelist10, filelist11, filelist12, filelist13, filelist14, filelist15, filelist16, filelist17, filelist18, filelist19, filelist20, filelist21, filelist22, filelist23]

Calcula los promedios de todas las imágenes

In [ ]:
print('[RGB promedio] [RGB teórico]')
subt = []
for i in range(len(l1)):
    subt.append(l1[i]) # agrega el elemento i a la lista l1 para luego comparar con el de la lista l2 
    subt.append(l2[i])
    mean = list(np.mean(subt, axis=0)) # promedio entre los rgb de todas las imagenes
    filelists[i].append(mean)
    print('El promedio de todas las imagenes para el color %d es:'%i, [int(round(filelists[i][0][0])),int(round(filelists[i][0][1])), int(round(filelists[i][0][2]))], [teorico[i][0],teorico[i][1],teorico[i][2]])
    subt.clear()    

[RGB promedio] [RGB teórico]
El promedio de todas las imagenes para el color 0 es: [179, 90, 61] [115, 82, 68]
El promedio de todas las imagenes para el color 1 es: [228, 213, 206] [194, 150, 130]
El promedio de todas las imagenes para el color 2 es: [161, 165, 190] [98, 122, 157]
El promedio de todas las imagenes para el color 3 es: [137, 150, 35] [87, 108, 67]
El promedio de todas las imagenes para el color 4 es: [226, 213, 232] [133, 128, 177]
El promedio de todas las imagenes para el color 5 es: [240, 243, 241] [103, 189, 170]
El promedio de todas las imagenes para el color 6 es: [214, 146, 2] [214, 126, 44]
El promedio de todas las imagenes para el color 7 es: [79, 87, 165] [80, 91, 166]
El promedio de todas las imagenes para el color 8 es: [205, 108, 132] [193, 90, 99]
El promedio de todas las imagenes para el color 9 es: [106, 18, 114] [94, 60, 108]
El promedio de todas las imagenes para el color 10 es: [220, 224, 163] [157, 188, 64]
El promedio de todas las imagenes para el col

Para el error vamos a analizar el archivo original que se tomó para imprimir el *ColorChecker* en [https://www.xrite.com/categories/calibration-profiling/colorchecker-classic](https://www.xrite.com/categories/calibration-profiling/colorchecker-classic)

In [ ]:
infog = files.upload()

Saving colorchecker-classic_01.png to colorchecker-classic_01.png


In [ ]:
imgcc = Image.open('colorchecker-classic_01.png')#imagenes ya cortadas y calibradas
#png = Image.open('prueba.png')

unitlenght = 70 #distancia entre centros
lstcc= [] # listas para guardar los valores rgb y luego compararlos, para obtener el promedio general
lcc = []

for j in range (0,4,1): #corre por cada fila (4 en total)
    for i in range (0,6,1): #corre por cada columna (6 en total) para 24 cuadros
        lenght = unitlenght*i #distancia entre centro 0 y centro cuadro i
        for a in range (33, 53, 2): #definir coordenadas columna cuadro 0
            a1 = a+lenght #correr horizontalmente del cuadro 0 -->
            for b in range(39+j*unitlenght,59+j*unitlenght,2): #definir coordenada fila cuadro 0 y correr verticalmente (abajo)
                lstcc.append(imgcc.getpixel((a1,b)))
        meancc=list(np.mean(lstcc, axis=0)) #promediar la lista
        lcc.append(meancc)
        lstcc.clear() #limpiar la lista para el siguiente cuadro

#print(lcc[1][0])
#for i in range(len(lcc)):
    #print('El promedio rgbcc de %d es:'%i,[int(round(lcc[i][0])),int(round(lcc[i][1])), int(round(lcc[i][2]))], [teorico[i][0],teorico[i][1],teorico[i][2]])

#Comparación de colores 
Los errores se expresan con el valor de wikipedia, el de la imagen original y el de las fotos (el experimental)

In [ ]:
print ('Color  : \t Teorico-Experimental \t Teorico-Descarga \t Experimental-Descarga')
for i in range(0,24):
    Real = sRGBColor(teorico[i][0]/255,teorico[i][1]/255,teorico[i][2]/255)
    CIEReal = convert_color(Real, LabColor)
    RGB = sRGBColor(int(round(filelists[i][0][0]))/255,int(round(filelists[i][0][1]))/255,int(round(filelists[i][0][2]))/255)
    CIE = convert_color(RGB, LabColor)
    delta_e = delta_e_cie2000(CIEReal, CIE);
    #print('La diferencia para el color %i de wikipedia y datos experimentales = ' %i, delta_e)
    RGBcc = sRGBColor(int(round(lcc[i][0]))/255,int(round(lcc[i][1]))/255,int(round(lcc[i][2]))/255)
    CIEcc = convert_color(RGBcc, LabColor)
    delta_ecc = delta_e_cie2000(CIEReal, CIEcc);
    #print('La diferencia para el color %i de wikipedia e imagen descargada = ' %i, delta_ecc)
    delta_eccexp = delta_e_cie2000(CIE,CIEcc);
    #print('La diferencia para el color %i con datos experimental e imagen descargada es = ' %i, delta_eccexp)
    print ('Color %i: \t %f \t\t %f \t\t %f' %(i, delta_e, delta_ecc, delta_eccexp))

Color  : 	 Teorico-Experimental 	 Teorico-Descarga 	 Experimental-Descarga
Color 0: 	 15.499157 		 15.464878 		 2.626760
Color 1: 	 18.101182 		 16.388891 		 7.868099
Color 2: 	 17.260334 		 19.102394 		 9.916648
Color 3: 	 20.971199 		 17.225893 		 7.307137
Color 4: 	 25.948843 		 20.260887 		 8.094493
Color 5: 	 26.211357 		 15.659697 		 14.883034
Color 6: 	 10.774519 		 8.532032 		 10.897552
Color 7: 	 1.371259 		 16.873100 		 18.071328
Color 8: 	 8.376856 		 14.542249 		 10.255724
Color 9: 	 10.011960 		 11.242073 		 2.331936
Color 10: 	 15.168603 		 12.568130 		 14.594694
Color 11: 	 12.000126 		 8.555413 		 8.698805
Color 12: 	 11.983831 		 8.692233 		 13.067881
Color 13: 	 4.584816 		 19.923941 		 19.676328
Color 14: 	 0.978985 		 15.988307 		 15.801255
Color 15: 	 9.303652 		 7.296129 		 14.206557
Color 16: 	 6.010235 		 18.358030 		 13.934045
Color 17: 	 6.931295 		 18.968061 		 16.708757
Color 18: 	 20.004739 		 1.353932 		 21.259392
Color 19: 	 9.329130 		 9.088422 		 18.134